In [1]:
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import numpy as np
import pandas as pd
import requests
import json
import datetime
import plotly.figure_factory as FF
from chart_studio.plotly import iplot
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from PIL import Image
from mpl_finance import candlestick_ohlc

from bs4 import BeautifulSoup as bs

batch_size = 32
img_height = 180
img_width = 180

c:\users\rob\trading\venv_38\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [2]:
def stochastic(row):
    return ((row['close']-row['14_min'])/(row['14_max']-row['14_min']))*100


def gain(row):
    if row['change'] >= 0:
        return row['change']
    else:
        return 0

def loss(row):
    if row['change']<= 0:
        return row['change']
    else:
        return 0
    
def rsi(row):
    return 100 - (100 / (1+((row['prev_avg_gain']*13)+row['gain'])/-((row['prev_avg_loss']*13)+row['loss'])))

In [11]:
stock_dict = {}

stocks = ['AAPL','BAC','TSLA','MSFT','FB','CSCO','ROKU','SPY','QQQ','IWM', 'SQ', 'INTC', 'PAYC','NVDA','WFC', 'GE', 'F']

for stock in stocks:
    r = requests.get(f"https://finance.yahoo.com/quote/{stock}/history?p={stock}")
    soup = bs(r.content)
    scripts = soup.find_all('script')
    pr = str(scripts[52]).split('root.App.main = ')[1]
    pr = pr.split(';\n}(this));\n</script>')[0]
    fin_data = json.loads(pr)
    new_df = pd.DataFrame(fin_data['context']['dispatcher']['stores']['HistoricalPriceStore']['prices'])
    if 'type' in new_df.columns:
        new_df = new_df[new_df['type']!='DIVIDEND']
    new_df.sort_values('date', inplace=True)
    new_df.reset_index(inplace=True)
    new_df.drop('index',axis=1,inplace=True)
    stock_dict[stock] = new_df

In [12]:
rsi_below_40 = tf.keras.models.load_model('rsi_below_60_model/')
rsi_above_60 = tf.keras.models.load_model('rsi_above_60_model/')
macd_below_0 = tf.keras.models.load_model('macd_below_0_model/')
macd_above_0 = tf.keras.models.load_model('macd_above_0_model/')

In [13]:
class_names = ['rsi_gain', 'rsi_loss']

for stock in stock_dict:
    df = stock_dict[stock]
    df.reset_index(inplace=True)
    df['change'] = df['close'] - df['close'].shift(1)
    df['ema_26'] = df['close'].ewm(span=26).mean()
    df['ema_12'] = df['close'].ewm(span=12).mean()
    df['macd'] = df['ema_12'] - df['ema_26']
    df['signal'] = df['macd'].ewm(9).mean()
    df['gain'] = df.apply(gain, axis = 1)
    df['loss'] = df.apply(loss, axis = 1)
    df['avg_gain'] = df['gain'].rolling(14).mean()
    df['avg_loss'] = df['loss']. rolling(14).mean()
    df['prev_avg_gain'] = df['avg_gain'].shift(1)
    df['prev_avg_loss'] = df['avg_loss'].shift(1)
    df['ma_20'] = df['close'].rolling(20).mean()
    df['ma_9'] = df['close'].rolling(9).mean()
    df['ma_50'] = df['close'].rolling(50).mean()
    df['14_min'] = df['low'].rolling(14).min()
    df['14_max'] = df['high'].rolling(14).max()
    df['stdev'] = abs(df['close'].rolling(20).std())
    df['upperband'] = df['ma_20'] + (df['stdev']*2)
    df['lowerband'] = df['ma_20'] - (df['stdev']*2)
    df['stochastic'] = df.apply(stochastic, axis = 1)
    df['rsi'] = df.apply(rsi, axis = 1)
    
    fig, ax = plt.subplots()
    ax.plot(df.iloc[-20:]['index'], df.iloc[-20:]['rsi'], color = 'blue', linewidth = 4)
    ax.plot(df.iloc[-20:]['index'], df.iloc[-20:]['stochastic'], color = 'green', linewidth = 4)
    ax.axhline(y=70, color = 'black', linewidth = 4)
    ax.axhline(y=30, color = 'black', linewidth = 4)


    ax.axis('off')
    fig.savefig(f'rsi_stoch_images/{stock}_rsi.jpeg')
    plt.close(fig)

    fig, ax = plt.subplots()
    ax.plot(df.iloc[-20:]['index'], df.iloc[-20:]['macd'], color = 'blue', linewidth = 4)
    ax.plot(df.iloc[-20:]['index'], df.iloc[-20:]['signal'], color = 'green', linewidth = 4)
    ax.axhline(y=0, color = 'black', linewidth = 4)


    ax.axis('off')
    fig.savefig(f'rsi_stoch_images/{stock}_macd.jpeg')
    plt.close(fig)
    
    

In [14]:
for stock in stocks:
    print(stock, stock_dict[stock].iloc[-1]['rsi'])
    
    img = keras.preprocessing.image.load_img(
        f"rsi_stoch_images/{stock}_rsi.jpeg", target_size=(img_height, img_width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    if stock_dict[stock].iloc[-1]['rsi'] < 50:
        predictions = rsi_below_40.predict(img_array)
        score = tf.nn.softmax(predictions[0])

        print(stock,"\n",
            "This image most likely belongs to {} with a {:.2f} percent confidence."
            .format(class_names[np.argmax(score)], 100 * np.max(score))
        )

    elif stock_dict[stock].iloc[-1]['rsi'] > 50:
        predictions = rsi_above_60.predict(img_array)
        score = tf.nn.softmax(predictions[0])

        print(stock,"\n",
            "This image most likely belongs to {} with a {:.2f} percent confidence."
            .format(class_names[np.argmax(score)], 100 * np.max(score))
        )
        
        
        
    img = keras.preprocessing.image.load_img(
        f"rsi_stoch_images/{stock}_macd.jpeg", target_size=(img_height, img_width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    if stock_dict[stock].iloc[-1]['macd'] < 0:
        predictions = macd_below_0.predict(img_array)
        score = tf.nn.softmax(predictions[0])

        print(stock,"\n",
            "This MACD image most likely belongs to {} with a {:.2f} percent confidence."
            .format(class_names[np.argmax(score)], 100 * np.max(score))
        )

    elif stock_dict[stock].iloc[-1]['macd'] > 0:
        predictions = macd_above_0.predict(img_array)
        score = tf.nn.softmax(predictions[0])

        print("This MACD image most likely belongs to {} with a {:.2f} percent confidence."
            .format(class_names[np.argmax(score)], 100 * np.max(score))
        )



AAPL 34.0374124343943
AAPL 
 This image most likely belongs to rsi_loss with a 72.76 percent confidence.
This MACD image most likely belongs to rsi_loss with a 83.73 percent confidence.
BAC 60.639929404387786
BAC 
 This image most likely belongs to rsi_gain with a 87.35 percent confidence.
This MACD image most likely belongs to rsi_gain with a 99.55 percent confidence.
TSLA 33.13260549071089
TSLA 
 This image most likely belongs to rsi_gain with a 99.37 percent confidence.
TSLA 
 This MACD image most likely belongs to rsi_loss with a 100.00 percent confidence.
MSFT 34.51174712349889
MSFT 
 This image most likely belongs to rsi_gain with a 97.90 percent confidence.
This MACD image most likely belongs to rsi_gain with a 84.25 percent confidence.
FB 56.03823254210688
FB 
 This image most likely belongs to rsi_loss with a 99.78 percent confidence.
This MACD image most likely belongs to rsi_gain with a 87.76 percent confidence.
CSCO 42.13626416117461
CSCO 
 This image most likely belongs to

In [7]:
api_token = 'sk_f491c9326e6a4b79a7ed5b1689c178f3'
token = 'pk_36b1f2b4c74a4c86bec74d5e48fb2517'

r = requests.get(f'https://cloud.iexapis.com/stable/stock/spy/chart/5dm?token={api_token}')

pr = json.loads(r.content)
df = pd.DataFrame(pr)

def future_max_price(row):
    i = int(row['index'])
    try:
        return max(df.iloc[i+1]['high'],df.iloc[i+2]['high'],df.iloc[i+3]['high'],df.iloc[i+4]['high'],df.iloc[i+5]['high'],df.iloc[i+6]['high'],df.iloc[i+7]['high'],df.iloc[i+8]['high'],df.iloc[i+9]['high'],df.iloc[i+10]['high'])
    except:
        return None
    
def future_min_price(row):
    i = int(row['index'])
    try:
        return min(df.iloc[i+1]['low'],df.iloc[i+2]['low'],df.iloc[i+3]['low'],df.iloc[i+4]['low'],df.iloc[i+5]['low'],df.iloc[i+6]['low'],df.iloc[i+7]['low'],df.iloc[i+8]['low'],df.iloc[i+9]['low'],df.iloc[i+10]['low'])
    except:
        return None
    
def stochastic(row):
    return ((row['close']-row['14_min'])/(row['14_max']-row['14_min']))*100


def gain(row):
    if row['change'] >= 0:
        return row['change']
    else:
        return 0

def loss(row):
    if row['change']<= 0:
        return row['change']
    else:
        return 0
    
def rsi(row):
    return 100 - (100 / (1+((row['prev_avg_gain']*13)+row['gain'])/-((row['prev_avg_loss']*13)+row['loss'])))
    

In [8]:
df.reset_index(inplace=True)

In [9]:
df['change'] = df['close'] - df['close'].shift(1)
df['gain'] = df.apply(gain, axis = 1)
df['loss'] = df.apply(loss, axis = 1)
df['avg_gain'] = df['gain'].rolling(14).mean()
df['avg_loss'] = df['loss']. rolling(14).mean()
df['ema_26'] = df['close'].ewm(span=26).mean()
df['ema_12'] = df['close'].ewm(span=12).mean()
df['macd'] = df['ema_12'] - df['ema_26']
df['signal'] = df['macd'].ewm(span=9).mean()
df['prev_avg_gain'] = df['avg_gain'].shift(1)
df['prev_avg_loss'] = df['avg_loss'].shift(1)
df['ma_20'] = df['close'].rolling(20).mean()
df['ma_9'] = df['close'].rolling(9).mean()
df['ma_50'] = df['close'].rolling(50).mean()
df['14_min'] = df['close'].rolling(14).min()
df['14_max'] = df['close'].rolling(14).max()
df['stdev'] = abs(df['close'].rolling(20).std())
df['upperband'] = df['ma_20'] + (df['stdev']*2)
df['lowerband'] = df['ma_20'] - (df['stdev']*2)
df['future_max_30'] = df.apply(future_max_price,axis=1)
df['future_min_30'] = df.apply(future_min_price,axis=1)
df['max_change'] = df['future_max_30'] - df['close']
df['min_change'] = df['future_min_30'] - df['close']
df['stochastic'] = df.apply(stochastic, axis = 1)
df['rsi'] = df.apply(rsi, axis = 1)

In [10]:
for i in df.index[27:190]:
    end = i
    fig, ax = plt.subplots()
    ax.plot(df.loc[end-20:end]['index'], df.loc[end-20:end]['rsi'], color = 'blue', linewidth = 4)
    ax.plot(df.loc[end-20:end]['index'], df.loc[end-20:end]['stochastic'], color = 'green', linewidth = 4)
    ax.axhline(y=70, color = 'black', linewidth = 4)
    ax.axhline(y=30, color = 'black', linewidth = 4)

    ax.axis('off')
    fig.savefig(f'spy_images/{i}_spy_rsi.jpeg')
    plt.close(fig)
    
    fig, ax = plt.subplots()
    ax.plot(df.loc[end-20:end]['index'], df.loc[end-20:end]['macd'], color = 'blue', linewidth = 4)
    ax.plot(df.loc[end-20:end]['index'], df.loc[end-20:end]['signal'], color = 'green', linewidth = 4)
    ax.axhline(y=0, color = 'black', linewidth = 4)

    ax.axis('off')
    fig.savefig(f'spy_images/{i}_spy_macd.jpeg')
    plt.close(fig)
    
    

In [114]:
buys = []
sells = []
long = False
enter = 0
longs = []
long_trades = []
short = False
shorts = []
short_trades = []
for i in df.index[27:190]:
    preds = ['none','none']
    img = keras.preprocessing.image.load_img(
        f"spy_images/{i}_spy_rsi.jpeg", target_size=(img_height, img_width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    if df.iloc[i]['rsi'] < 50:
        predictions = rsi_below_40.predict(img_array)
        score = tf.nn.softmax(predictions[0])

        preds[0] = class_names[np.argmax(score)]

    elif df.iloc[i]['rsi'] > 50:
        predictions = rsi_above_60.predict(img_array)
        score = tf.nn.softmax(predictions[0])

        preds[0] = class_names[np.argmax(score)]
        
    img = keras.preprocessing.image.load_img(
        f"spy_images/{i}_spy_macd.jpeg", target_size=(img_height, img_width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    if df.iloc[i]['macd'] < 0:
        predictions = macd_below_0.predict(img_array)
        score = tf.nn.softmax(predictions[0])

        preds[1] = class_names[np.argmax(score)]

    elif df.iloc[i]['macd'] > 0:
        predictions = macd_above_0.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        preds[1] = class_names[np.argmax(score)]
        
    if preds[0] == 'rsi_gain' and preds[1] == 'rsi_gain':
        buys.append((df.iloc[i]['close'],df.iloc[i]['future_max_30'],df.iloc[i]['future_min_30'],df.iloc[i]['future_max_30']-df.iloc[i]['close'],df.iloc[i]['future_min_30']-df.iloc[i]['close']))
    
        
    if preds[0] == 'rsi_loss' and preds[1] == 'rsi_loss':
        sells.append((df.iloc[i]['close'],df.iloc[i]['future_max_30'],df.iloc[i]['future_min_30'],df.iloc[i]['future_max_30']-df.iloc[i]['close'],df.iloc[i]['future_min_30']-df.iloc[i]['close']))
    
    if long and df.iloc[i]['low'] - enter <-.40:
        long = False
        longs.append('loss')
        long_trades.append((enter_i, enter, i))
    
    if long and df.iloc[i]['high'] - enter > .80:
        long = False
        longs.append('win')
        long_trades.append((enter_i, enter, i))
    if preds[0] == 'rsi_gain' and preds[1] == 'rsi_gain' and not long and not short and df.iloc[i]['rsi'] >50 and df.iloc[i]['macd']>0:
        long = True
        enter = df.iloc[i]['close']
        enter_i = i
 
    if short and df.iloc[i]['low'] - enter <-.80:
        short = False
        shorts.append('win')
        short_trades.append((enter_i, enter, i))
    
    if short and df.iloc[i]['high'] - enter > .40:
        short = False
        shorts.append('loss')
        short_trades.append((enter_i, enter, i))

    if preds[0] == 'rsi_loss' and preds[1] == 'rsi_loss' and not long and not short and df.iloc[i]['macd']<0 and df.iloc[i]['rsi']<50:
        short = True
        enter = df.iloc[i]['close']
        enter_i = i
        

In [21]:
buys = np.array(buys)
buy_gains = buys[:,3]
buy_losses = buys[:,4]

In [22]:
sells = np.array(sells)
sell_gains = sells[:,3]
sell_losses = buys[:,4]

In [23]:
buy_gains

array([1.265, 1.82 , 1.855, 1.84 , 1.93 , 1.14 , 0.335, 0.225, 0.73 ,
       0.655, 1.07 , 0.1  , 0.98 , 1.31 , 1.24 , 1.43 , 1.175, 0.845,
       0.55 , 0.35 , 0.28 , 0.395, 0.215, 0.275, 0.33 , 0.25 ,   nan,
       1.205, 1.345, 0.81 , 0.43 , 0.59 , 1.   , 0.59 , 0.13 , 0.225,
       0.38 , 0.64 , 0.42 , 0.03 , 0.12 , 0.47 , 0.215, 0.17 , 0.185,
       0.475, 2.305, 4.2  , 1.365, 1.21 , 0.555, 0.53 , 0.245, 0.735,
       1.07 , 0.17 , 0.6  , 0.59 , 0.5  , 0.295,   nan])

In [24]:
buy_losses

array([-0.235, -0.33 , -0.295, -0.31 , -0.22 , -0.11 , -0.665, -0.775,
       -0.265, -0.125, -0.105, -0.48 , -0.39 , -0.29 , -0.36 ,  0.   ,
       -0.11 , -0.195, -0.49 , -0.13 , -0.2  , -0.06 , -0.345, -0.285,
       -0.325, -0.405,    nan, -0.045, -0.065, -0.14 , -0.815, -0.775,
       -0.365, -0.58 , -1.04 , -0.945, -0.79 , -0.53 , -0.7  , -1.09 ,
       -0.68 , -1.89 , -2.145, -2.085, -2.07 , -1.21 , -1.185,  2.46 ,
       -0.375,  0.06 , -0.345, -0.39 , -0.765, -0.405, -0.09 , -0.53 ,
       -0.1  , -0.08 , -0.1  , -0.585,    nan])

In [25]:
buy_gains.mean()

nan

In [26]:
buy_losses.mean()

nan

In [27]:
buy_gains.sum()

nan

In [28]:
buy_losses.sum()

nan

In [29]:
sell_gains[:7].mean()

0.5621428571428656

In [30]:
sell_losses[:7].mean()

-0.3092857142857172

In [31]:
sell_gains[:7].sum()

3.935000000000059

In [32]:
sell_losses[:7].sum()

-2.1650000000000205

In [115]:
len([x for x in longs if x == 'win'])

8

In [116]:
len([x for x in longs if x == 'loss'])

7

In [117]:
long_trades

[(27, 409.745, 36),
 (41, 410.87, 45),
 (45, 410.47, 56),
 (63, 411.03, 65),
 (70, 410.94, 76),
 (76, 411.635, 79),
 (84, 411.86, 106),
 (108, 412.715, 112),
 (112, 413.39, 114),
 (114, 412.94, 119),
 (119, 413.81, 120),
 (120, 413.715, 122),
 (122, 413.3, 128),
 (160, 414.23, 162),
 (163, 414.885, 177)]

In [118]:
len([x for x in shorts if x == 'win'])

1

In [119]:
len([x for x in shorts if x == 'loss'])

1

In [107]:
short_trades

[]

In [39]:
df.iloc[188]

index                              188
date                        2021-04-15
minute                           14:50
label                            14:50
open                            415.37
high                           415.475
low                             415.19
close                           415.31
average                     415.332697
volume                            9308
notional                    3865918.82
numberOfTrades                     131
marketOpen                      415.36
marketHigh                      415.48
marketLow                       415.18
marketClose                     415.36
marketAverage               415.351117
marketVolume                    625587
marketNotional          259838210.7668
marketNumberOfTrades              4096
change                           -0.05
gain                               0.0
loss                             -0.05
avg_gain                        0.0625
avg_loss                     -0.070357
ema_26                   

In [14]:
mylist = ['hello','hi']

In [16]:
mylist.pop(1)

'hi'

In [17]:
mylist

['hello']

Successful combinations: 
Buy when RSI > 50 and MACD > 0
Combined with selling when MACD < 0